In [ ]:
mport tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
from PIL import Image, ImageTk
import pytesseract
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Point pytesseract to the tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load datasets
fake = pd.read_csv('Fake.csv')
real = pd.read_csv('True.csv')

# Assign labels
real['label'] = 1
fake['label'] = 0

# Combine datasets
news = pd.concat([fake, real], axis=0)

# Drop unnecessary columns
news = news.drop(['title', 'subject', 'date'], axis=1)

# Shuffle the data
news = news.sample(frac=1).reset_index(drop=True)

# Text preprocessing function
def wordopt(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply preprocessing to the text column
news['text'] = news['text'].apply(wordopt)

# Split the data into features and labels
x = news['text']
y = news['label']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Vectorize the text data
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Train a logistic regression model
LR = LogisticRegression()
LR.fit(xv_train, y_train)

# Function to output labels
def output_label(n):
    if n == 0:
        return "fake news"
    elif n == 1:
        return "genuine news"

# Function for manual testing
def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_lr = LR.predict(new_xv_test)
    return output_label(pred_lr[0])

# Create the main window
root = tk.Tk()
root.title("Fake News Detector")
root.geometry("600x600")

# Label for instructions
instructions = tk.Label(root, text="Choose an option to check if the news is fake or genuine:", font=('Arial', 14))
instructions.grid(row=0, column=0, columnspan=2, pady=10)

# Global variable to store extracted text
text_from_image = ""

# Function to upload image
def upload_image():
    global img, text_from_image
    file_path = filedialog.askopenfilename()
    if file_path:
        img = Image.open(file_path)
        img = img.resize((300, 300), Image.LANCZOS)
        img = ImageTk.PhotoImage(img)
        panel.config(image=img)
        panel.image = img
        text_from_image = pytesseract.image_to_string(Image.open(file_path))
        extracted_text.delete("1.0", tk.END)
        extracted_text.insert(tk.END, text_from_image)
        messagebox.showinfo("Extracted Text", "Text has been extracted from the image.")

# Function to predict if the text is fake or genuine
def predict():
    text_to_check = extracted_text.get("1.0", tk.END).strip()
    if text_to_check and text_to_check != "write something here":
        prediction = manual_testing(text_to_check)
        messagebox.showinfo("Prediction", f"The article is predicted to be: {prediction}")
    else:
        messagebox.showwarning("Input Error", "Please provide some text to check.")

# Frame for manual text input
text_frame = tk.Frame(root)
text_frame.grid(row=1, column=0, columnspan=2, pady=10)

# Label and scrolled text widget for manual text input
manual_text_label = tk.Label(text_frame, text="Or manually enter the text:", font=('Arial', 12))
manual_text_label.pack(anchor='w')

extracted_text = scrolledtext.ScrolledText(text_frame, wrap=tk.WORD, width=60, height=10, font=('Arial', 12))
extracted_text.insert(tk.END, "write something here")
extracted_text.pack(pady=5)

# Button to upload image
upload_button = tk.Button(root, text="Upload Image", command=upload_image, font=('Arial', 12))
upload_button.grid(row=2, column=0, pady=10)

# Panel to display the image
panel = tk.Label(root)
panel.grid(row=3, column=0, columnspan=2)

# Button to predict
predict_button = tk.Button(root, text="Predict", command=predict, font=('Arial', 12))
predict_button.grid(row=2, column=1, pady=10)

# Run the GUI main loop
root.mainloop()
